In [59]:
import sys
sys.path.append('../utils')

from fastai.vision.all import *
from fastai.vision.widgets import *
import fastai
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from augmentation import *


In [77]:
import warnings
warnings.filterwarnings('ignore')  # or 'once'  

In [60]:
# RUN_NAME_OF_MODEL_TO_LOAD = '20210210-1221 - arch=tf_efficientnet_b4_ns - samples=1800 frozen=1 epochs=40 bs=8 res=480'  # BEST 87%
# EPOCH_TO_LOAD=2

# RUN_NAME_OF_MODEL_TO_LOAD = '20210209-0234 - arch=tf_efficientnet_b4_ns - samples=1800 frozen=1 epochs=40 bs=7 res=456'  # 83%
# RUN_NAME_OF_MODEL_TO_LOAD = '20210214-1340 - arch=<function xse_resnext34 at 0x7fed41524d30> - samples=1800 frozen=1 epochs=60 bs=14 res=420'
# EPOCH_TO_LOAD=15

#f'20210209-0234 - arch=tf_efficientnet_b4_ns - samples=1800 frozen=1 epochs=40 bs=7 res=456'
# EPOCH_TO_LOAD = 2

# EPOCH_TO_LOAD = 8
# RUN_NAME_OF_MODEL_TO_LOAD = '20210215-0355 - arch=<function xse_resnext34 at 0x7f6ed4c51d30> - samples=1800 frozen=1 epochs=60 bs=32 res=250'




RESOLUTION=300
BATCH_SIZE = 32
SAMPLE_SIZE = 2000

path = '../../data'
sub = 'train_images'  # 'test_images'
# dataset_path = Path(path)

In [101]:
# sample_df = pd.read_csv(f'{path}/sample_submission.csv')
# sample_df = pd.read_csv(f'{path}/all_test_files.csv')
sample_df = pd.read_csv('../DenseNet/input_persist/20210223-0550 - arch=densenetblur121d - samples=-1 frozen=1 epochs=5 bs=32 res=300_ensemble_train_df.csv', index_col=0)

In [102]:
## sample_df['image_id'] =  f'{path}/' + sample_df['image_id'].astype(str)
# cols = sample_df.columns.tolist()
# cols = cols[::-1]  # reverse the columns
# sample_df = sample_df[cols]
##sample_df[0]

In [103]:
s = sample_df.image_id.str.split('/')
s = pd.Series([line[-1] for line in s], name='image_id')
sample_df.loc[:, 'image_id'] = s

In [104]:
sample_df

,image_id,label
0,1745816126.jpg,4
1,3307346645.jpg,1
2,train-cbsd-1025.jpg,1
3,train-cmd-1233.jpg,3
4,3940020212.jpg,3
...,...,...
3995,2257519479.jpg,3
3996,2889971119.jpg,3
3997,1274100886.jpg,3
3998,1555851966.jpg,2


In [105]:
# aug_tfms =  aug_transforms(size=RESOLUTION, pad_mode='reflection', do_flip=True, batch=False, 
#                                p_affine=0.7, max_rotate=45, max_warp=0, min_zoom=1.0, max_zoom=2.5, 
#                                mult=3,  p_lighting=0.5, max_lighting=0.1, min_scale=0.75) #,xtra_tfms=[sha])

sat = Saturation(p=0.7, max_lighting=0.2)

# batch_tfms = [ Normalize.from_stats(*imagenet_stats), sat]
# batch_tfms = [ *aug_tfms, sat, Normalize.from_stats(*imagenet_stats)]#, sha]
batch_tfms = [ Normalize.from_stats(*imagenet_stats) ]#, sha]

# item_tfms=RandomResizedCrop(RESOLUTION)
item_tfms = AlbumentationsTransform(get_train_aug(RESOLUTION), get_valid_aug(RESOLUTION))

# item_tfms=Resize(RESOLUTION, method='bilinear', pad_mode='zeros')
#     item_tfms=RatioResize(RESOLUTION)

dls = ImageDataLoaders.from_df(sample_df, folder=f'../../data/{sub}', seed=42, label_col=1, fn_col=0, 
                               batch_tfms=batch_tfms, bs=BATCH_SIZE, item_tfms=item_tfms)
test_dl = dls.test_dl(sample_df)

In [65]:
dls.test_dl

<bound method DataLoaders.test_dl of <fastai.data.core.DataLoaders object at 0x7fd45b898370>>

In [66]:
def get_label(o):
    fname = o.name
    
    if name in sample_df.image_id.values:
        return sample_df[sample_df.image_id == fname].label.values[0]

In [67]:
# model = load_learner(f'../models/{RUN_NAME_OF_MODEL_TO_LOAD}.pkl', cpu=False)
# import timm
# model = timm_learner(dls,
#                     'tf_efficientnet_b4_ns',
#                      opt_func=ranger,
#                      loss_func=LabelSmoothingCrossEntropy(),
#           #           cbs=cbs,
#                      metrics = [accuracy]).to_fp16()


# ARCH = xse_resnext34(pretrained=True, act_cls=MishAuto, sa=True)

# model = Learner(dls, ARCH, 
#                     opt_func=ranger,
#                     loss_func=LabelSmoothingCrossEntropyFlat(), 
#                      metrics=[accuracy])


model=load_learner('../DenseNet/models/STAGE1_20210223-0550 - arch=densenetblur121d - samples=-1 frozen=1 epochs=5 bs=32 res=300.pkl', cpu=False)


# model = convert_MP_to_blurMP(model, nn.MaxPool2d)  # doesn't exist!
# model.model = convert_act_cls(model.model, nn.ReLU, MishAuto())  # very expensive so we only use it on the last layers
# model.model = convert_act_cls(model.model, nn.SiLU, SwishAuto())  # replace with a lower memory version
# model.model[0] = convert_seq_chkpt(model.model[0], nn.Sequential) #, 0)  # wrap all BODY sequentials to use Gradient Checkpointing
# model.model[0] = nn.Sequential(CheckpointModule(model.model[0]))  # hack in the final checkpointmodule for the base sequence

# @TODO WATCHT HIS
# model = model.to_fp16()  # convert to 16bit

# load_model(f'models/{RUN_NAME_OF_MODEL_TO_LOAD}_{EPOCH_TO_LOAD}.pth', model, opt=ranger, with_opt=False)

# model = model.to_fp32()


load_model('../DenseNet/models/20210223-0550 - arch=densenetblur121d - samples=-1 frozen=1 epochs=5 bs=32 res=300_split_8_epoch25_acc0.88.pth', model, opt=ranger)

/home/frank/miniconda3/envs/learn/lib/python3.8/site-packages/fastai/learner.py:55: UserWarning: Could not load the optimizer state.
  if with_opt: warn("Could not load the optimizer state.")


In [68]:
model.cbs

(#11) [TrainEvalCallback,Recorder,ProgressCallback,CutMix,SaveModelCallback,ShowGraphCallback,EarlyStoppingCallback,GradientClip,GradientAccumulation,ModelToHalf...]

In [76]:
def remove_callbacks():
    model.remove_cbs(CutMix)
    model.remove_cbs(ShowGraphCallback)
    model.remove_cbs(EarlyStoppingCallback)
    model.remove_cbs(SaveModelCallback)
#     model.cbs

In [70]:
remove_callbacks()
model.cbs

(#7) [TrainEvalCallback,Recorder,ProgressCallback,GradientClip,GradientAccumulation,ModelToHalf,NonNativeMixedPrecision]

In [71]:
# preds, something = model.tta(dl=test_dl, n=5, use_max=False, beta=0.18)

In [72]:
# test_dl.show_batch()

In [73]:
preds, something = model.tta(dl=test_dl, n=3)  #, use_max=False, beta=0.1)
print(classification_report(sample_df.iloc[:,1], preds.argmax(dim=-1).numpy()))  # TTA

/home/frank/miniconda3/envs/learn/lib/python3.8/site-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


              precision    recall  f1-score   support

           0       0.65      0.54      0.59       155
           1       0.89      0.79      0.84       363
           2       0.87      0.69      0.77       316
           3       0.92      0.98      0.95      1582
           4       0.66      0.72      0.69       289

    accuracy                           0.87      2705
   macro avg       0.80      0.74      0.77      2705
weighted avg       0.87      0.87      0.87      2705



In [75]:

preds2 = model.get_preds(dl=test_dl)
print(classification_report(sample_df.iloc[:,1], preds2[0].argmax(dim=-1).numpy()))  # straight

/home/frank/miniconda3/envs/learn/lib/python3.8/site-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


              precision    recall  f1-score   support

           0       0.65      0.54      0.59       155
           1       0.85      0.79      0.82       363
           2       0.83      0.72      0.77       316
           3       0.93      0.96      0.94      1582
           4       0.64      0.73      0.69       289

    accuracy                           0.86      2705
   macro avg       0.78      0.75      0.76      2705
weighted avg       0.86      0.86      0.86      2705



In [ ]:
# print(classification_report(sample_df.iloc[:,0], ((preds + preds2[0]).argmax(dim=-1).numpy())))  # average

In [ ]:
# print(classification_report(sample_df.iloc[:,0], (torch.maximum(preds,preds2[0]).argmax(dim=-1).numpy())))  # max

In [ ]:
# Grid Searching

# # a_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.999]
# a_list=[1]
# # a_list = np.arange(0.5,1.0,0.1)
# #[1] #[0,45,90,135]  #[750, 800, 850, 900]# list(np.arange(0, 5, 0.5)) # [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# scores_gs = []

# for VARIABLE in a_list:
#     aug_tfms =  aug_transforms(size=RESOLUTION, pad_mode='reflection', do_flip=True, batch=False, 
#                                    p_affine=0.7, max_rotate=45, max_warp=0, min_zoom=1.0, max_zoom=2.5, 
#                                    mult=3,  p_lighting=0.5, max_lighting=0.1, min_scale=0.75) #,xtra_tfms=[sha])

#     sat = Saturation(p=0.7, max_lighting=0.2)

#     # batch_tfms = [ Normalize.from_stats(*imagenet_stats), sat]
#     batch_tfms = [ *aug_tfms, sat, Normalize.from_stats(*imagenet_stats)]#, sha]

# #     item_tfms=RandomResizedCrop(RESOLUTION)
#     item_tfms=Resize(RESOLUTION, method='bilinear', pad_mode='zeros')
# #     item_tfms=RatioResize(RESOLUTION)

#     dls = ImageDataLoaders.from_df(sample_df, folder=f'../data/{sub}', seed=42, label_col = 0, fn_col=1, 
#                                    batch_tfms=batch_tfms, bs=BATCH_SIZE, item_tfms=item_tfms)
#     test_dl = dls.test_dl(sample_df)

#     preds, something = model.tta(dl=test_dl, n=5, use_max=False, beta=0.18)
#     scores_gs.append(accuracy_score(sample_df.iloc[:,0], preds.argmax(dim=-1).numpy()))  # TTA
    
#     print(f'best SO FAR: {a_list[np.argmax(scores_gs)]}, at {scores_gs[np.argmax(scores_gs)]}')


# print(f'best n: {a_list[np.argmax(scores_gs)]}')
# print(a_list)
# print(scores_gs)

In [ ]:
# test_dl.show_batch()

In [106]:
all_preds = []
all_models = ['20210223-0550 - arch=densenetblur121d - samples=-1 frozen=1 epochs=5 bs=32 res=300_split_8_epoch25_acc0.88.pth',
              '20210223-0550 - arch=densenetblur121d - samples=-1 frozen=1 epochs=5 bs=32 res=300_split_6_epoch20_acc0.88.pth',
              '20210223-0550 - arch=densenetblur121d - samples=-1 frozen=1 epochs=5 bs=32 res=300_split_4_epoch25_acc0.88.pth',
              '20210223-0550 - arch=densenetblur121d - samples=-1 frozen=1 epochs=5 bs=32 res=300_split_2_epoch15_acc0.88.pth',
              '20210223-0550 - arch=densenetblur121d - samples=-1 frozen=1 epochs=5 bs=32 res=300_split_3_epoch20_acc0.86.pth']

for pth in all_models:
    load_model(f'../DenseNet/models/{pth}', model, opt=ranger, with_opt=False)
    remove_callbacks()
    preds, something = model.tta(dl=test_dl, n=3)  #, use_max=False, beta=0.1)
    print(classification_report(sample_df.iloc[:,1], preds.argmax(dim=-1).numpy()))  # TTA
    
    all_preds.append(preds)

              precision    recall  f1-score   support

           0       0.68      0.56      0.61       230
           1       0.85      0.81      0.83       537
           2       0.88      0.70      0.78       467
           3       0.93      0.97      0.95      2338
           4       0.70      0.80      0.74       428

    accuracy                           0.88      4000
   macro avg       0.81      0.77      0.78      4000
weighted avg       0.88      0.88      0.87      4000



              precision    recall  f1-score   support

           0       0.64      0.65      0.65       230
           1       0.84      0.80      0.82       537
           2       0.84      0.76      0.80       467
           3       0.95      0.96      0.95      2338
           4       0.72      0.79      0.75       428

    accuracy                           0.88      4000
   macro avg       0.80      0.79      0.79      4000
weighted avg       0.88      0.88      0.88      4000



              precision    recall  f1-score   support

           0       0.67      0.59      0.62       230
           1       0.80      0.83      0.82       537
           2       0.88      0.73      0.80       467
           3       0.94      0.97      0.96      2338
           4       0.73      0.77      0.75       428

    accuracy                           0.88      4000
   macro avg       0.80      0.78      0.79      4000
weighted avg       0.88      0.88      0.88      4000



              precision    recall  f1-score   support

           0       0.62      0.58      0.60       230
           1       0.83      0.82      0.82       537
           2       0.88      0.75      0.81       467
           3       0.94      0.97      0.95      2338
           4       0.73      0.73      0.73       428

    accuracy                           0.88      4000
   macro avg       0.80      0.77      0.78      4000
weighted avg       0.87      0.88      0.87      4000



              precision    recall  f1-score   support

           0       0.60      0.72      0.66       230
           1       0.82      0.77      0.79       537
           2       0.78      0.82      0.80       467
           3       0.98      0.91      0.94      2338
           4       0.64      0.81      0.71       428

    accuracy                           0.86      4000
   macro avg       0.76      0.81      0.78      4000
weighted avg       0.87      0.86      0.86      4000



In [128]:
all_preds_en = all_preds.copy()


In [133]:
hat = all_preds_en[0]
for pred in all_preds_en[1:]:
  hat += pred
        

In [135]:
hat /= len(all_preds_en)

In [139]:
print(classification_report(sample_df.iloc[:,1], hat.argmax(dim=-1).numpy()))

              precision    recall  f1-score   support

           0       0.64      0.66      0.65       230
           1       0.84      0.83      0.84       537
           2       0.88      0.76      0.82       467
           3       0.96      0.96      0.96      2338
           4       0.73      0.79      0.76       428

    accuracy                           0.89      4000
   macro avg       0.81      0.80      0.80      4000
weighted avg       0.89      0.89      0.89      4000



In [132]:
all_preds_en

[tensor([[0.1426, 0.1343, 0.1043, 0.1981, 0.4208],
         [0.0433, 0.7201, 0.0163, 0.1292, 0.0911],
         [0.0876, 0.4870, 0.0429, 0.0834, 0.2991],
         ...,
         [0.0227, 0.0331, 0.0265, 0.8898, 0.0280],
         [0.0114, 0.0148, 0.9257, 0.0364, 0.0117],
         [0.0224, 0.0279, 0.0648, 0.8515, 0.0334]]),
 tensor([[0.1624, 0.1361, 0.0949, 0.2643, 0.3423],
         [0.0428, 0.6951, 0.0155, 0.1328, 0.1138],
         [0.0680, 0.5395, 0.0482, 0.0811, 0.2631],
         ...,
         [0.0243, 0.0350, 0.0231, 0.8886, 0.0290],
         [0.0120, 0.0112, 0.9295, 0.0382, 0.0091],
         [0.0195, 0.0253, 0.0373, 0.8906, 0.0273]]),
 tensor([[0.1231, 0.2378, 0.0611, 0.1623, 0.4157],
         [0.0233, 0.7165, 0.0166, 0.1716, 0.0720],
         [0.0530, 0.6822, 0.0327, 0.1102, 0.1219],
         ...,
         [0.0205, 0.0212, 0.0243, 0.9096, 0.0244],
         [0.0145, 0.0118, 0.9155, 0.0374, 0.0208],
         [0.0207, 0.0258, 0.1337, 0.7921, 0.0277]]),
 tensor([[0.1678, 0.1032, 0.0759, 

In [142]:
import pickle 

with open('all_preds_en.p', 'wb') as f:
    pickle.dump(all_preds_en, f)